# load mongo

In [2]:
import codecs
import sys
import os
import logging

logger = logging.getLogger(__name__)


In [3]:
sys.path.append("/Users/xszpo/Google Drive/DataScience/Projects/201907_xFlat_AWS_Scrapy/app_webscr_otodom")
import helpers as helpers

# run scrapy settings
%run /Users/xszpo/Google\ Drive/DataScience/Projects/201907_xFlat_AWS_Scrapy/app_webscr_otodom/scraper/settings.py


In [4]:
import pymongo
from pymongo.errors import ConnectionFailure, ServerSelectionTimeoutError, WriteError, WriteConcernError, WTimeoutError

class Mongo:
    """
    MongoDB methods
    """

    def __init__(self,
                 db_address='localhost',
                 db_port=27017,
                 db_username= '',
                 db_password=''
                 ):
        self.db_address = db_address
        self.db_port = db_port
        self.db_username = db_username
        self.db_password = db_password
        

    def mongodb_conn(self):
        """
        https://www.programiz.com/python-programming/property
        :return:
            pymongo conetion
        """

        try:
            conn = pymongo.MongoClient(self.db_address, self.db_port, serverSelectionTimeoutMS=5000)
            logger.info("Connected to MongoDB {}, address: {}:{}".format(conn.server_info()['version'],
                        self.db_address, self.db_port))
            return conn

        except (ConnectionFailure, ServerSelectionTimeoutError) as e:
            logger.error("pymongo.errors, Could not connect to server: %s" % e)

        except BaseException as e:
            logger.error("BaseException, something went wrong: %s" % e)


In [5]:
def mongodb_conn(db_address='localhost',db_port=27017,db_username= '',db_password=''):
    """
    https://www.programiz.com/python-programming/property
    :return:
        pymongo conetion
    """

    try:
        if db_username == '':
            conn = pymongo.MongoClient(host=db_address, port=db_port,serverSelectionTimeoutMS=5000)
        else:
            conn = pymongo.MongoClient(host=db_address, port=db_port,username=db_username,password=db_password,
                                   authSource='admin', authMechanism='SCRAM-SHA-256', serverSelectionTimeoutMS=5000)
        
        logger.info("Connected to MongoDB {}, address: {}:{}".format(conn.server_info()['version'],
                                                                     db_address, db_port))
        return conn

    except (ConnectionFailure, ServerSelectionTimeoutError) as e:
        logger.error("pymongo.errors, Could not connect to server: %s" % e)

    except BaseException as e:
        logger.error("BaseException, something went wrong: %s" % e)

In [6]:
conn = mongodb_conn(db_address=MONGO_ADDRESS,db_port=MONGO_PORT,
                    db_username= MONGO_USERNAME,db_password=MONGO_PASSWORD)

In [7]:
db =conn[MONGO_DBNAME][MONGO_COLL_OTODOM]

In [8]:
db.find({}).count()

/Users/xszpo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


0

In [9]:
globals().keys()


dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'codecs', 'sys', 'os', 'logging', 'logger', '_i2', '_i3', 'helpers', '__nonzero__', 'BOT_NAME', 'MAX_PAGES', 'SAVE_RESULTS', 'LOCAL_DATA_PATH', 'LOCAL_DATA_PATH_PREP', 'LOCAL_DATA_PATH_DICT', 'BUCKET_NAME', 'BUCKET_PREFIX_BSON', 'MONGO_ADDRESS', 'MONGO_PORT', 'MONGO_DBNAME', 'MONGO_COLL_OTODOM', 'MONGO_USERNAME', 'MONGO_PASSWORD', 'DOWNLOAD_IMAGES', 'SPIDER_MODULES', 'NEWSPIDER_MODULE', 'LOG_ENABLE', 'LOG_LEVEL', 'ROBOTSTXT_OBEY', 'DOWNLOAD_DELAY', 'RANDOMIZE_DOWNLOAD_DELAY', 'CONCURRENT_REQUESTS', 'COOKIES_ENABLED', '_i4', 'pymongo', 'ConnectionFailure', 'ServerSelectionTimeoutError', 'WriteError', 'WriteConcernError', 'WTimeoutError', 'Mongo', '_i5', 'mongodb_conn', '_i6', 'conn', '_i7', 'db', '_i8', '_8', '_i9'])

In [10]:
client = pymongo.MongoClient(host=MONGO_ADDRESS, port=MONGO_PORT,
                             username=MONGO_USERNAME,
                             password=MONGO_PASSWORD,
                             authSource='admin',
                             authMechanism='SCRAM-SHA-256')

In [11]:
db = client[MONGO_DBNAME][MONGO_COLL_OTODOM]

In [12]:
db

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', authmechanism='SCRAM-SHA-256'), 'OFFERS'), 'Otodom')

# put data in mongo

In [25]:
import sys
sys.path.append("/Users/xszpo/Google Drive/DataScience/Projects/201907_xFlat_AWS_Scrapy/app_webscr_otodom")
import helpers as helpers
import gc
import numpy as np
import json

def load_raw_data(file_name_list, file_path):
    
    columns = ['_id','name','location','flat_size', 'rooms','floor','price',
               'price_m2', 'market', 'number_of_floors', 'floor_attic','floor_basement','building_type', 
               'building_material', 'widows_type', 'heating_type', 'year_of_building','finishing_stage', 
               'rent_price', 'property_form', 'available_from','description','additional_info',
               'GC_latitude','GC_longitude', 'GC_boundingbox', 'GC_addr_house_number','GC_addr_road', 
               'GC_addr_neighbourhood', 'GC_addr_suburb','GC_addr_city', 'GC_addr_county', 'GC_addr_state', 
               'GC_addr_postcode','GC_addr_country', 'GC_addr_country_code','url','main_url','gallery',
               'img_gallery_strimg','tracking_id','download_date']
    
    doc_list = []
    
    for name in file_name_list:
        # LOAD DATA
        _tmp = helpers.scraper.read_bson_local(file_path, name+".bson")

        # PUT DATA INTO COLUMNS
        _tmp['_id'] = name        
        _tmp['GC_latitude'] = float(_tmp['geo_coordinates']['latitude'])
        _tmp['GC_longitude'] = float(_tmp['geo_coordinates']['longitude'])
        _tmp['GC_boundingbox'] = _tmp['geo_address_coordin']['@boundingbox']
        _tmp['GC_addr_house_number'] = _tmp['geo_address_text']['house_number'] if 'house_number' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_road'] = _tmp['geo_address_text']['road'] if 'road' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_neighbourhood'] = _tmp['geo_address_text']['neighbourhood'] if 'neighbourhood' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_suburb'] = _tmp['geo_address_text']['suburb'] if 'suburb' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_city'] = _tmp['geo_address_text']['city'] if 'city' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_county'] = _tmp['geo_address_text']['county'] if 'country' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_state'] = _tmp['geo_address_text']['state'] if 'state' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_postcode'] = _tmp['geo_address_text']['postcode'] if 'postcode' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_country'] = _tmp['geo_address_text']['country'] if 'country' in _tmp['geo_address_text'] else None
        _tmp['GC_addr_country_code'] = _tmp['geo_address_text']['country_code'] if 'country_code' in _tmp['geo_address_text'] else None
        _tmp['additional_info'] =_tmp['additional_info'].split("|")
        _ = _tmp.pop('geo_coordinates')
        _ = _tmp.pop('geo_address_coordin') 
        _ = _tmp.pop('price_per_square')

        # MODIFY DATA
        _tmp['flat_size'] = float(np.float32(helpers.scraper.digits_from_str(_tmp['flat_size']))) if _tmp['flat_size'] is not None else None
        _tmp['price'] = float(np.float32(helpers.scraper.digits_from_str(_tmp['price']))) if _tmp['price'] is not None else None
        _tmp['price_m2'] = helpers.scraper.digits_from_str(_tmp['price_m2']) if _tmp['price_m2'] is not None else None
        _tmp['rooms'] = int(helpers.scraper.digits_from_str(_tmp['rooms'])) if _tmp['rooms'] is not None else None
        _tmp['floor_attic'] = 1 if _tmp['floor']=='poddasze' else 0
        _tmp['floor_basement'] = 1 if _tmp['floor']=='suterena' else 0
        _tmp['floor'] = float(np.float32(helpers.scraper.convert_floor(_tmp['floor']))) if isinstance(_tmp['floor'], (str)) else None
        _tmp['number_of_floors'] = float(np.float32(_tmp['number_of_floors'])) if isinstance(_tmp['number_of_floors'], (str)) else None
        _tmp['year_of_building'] = float(np.float32(_tmp['year_of_building'])) if isinstance(_tmp['year_of_building'], (str)) else None
        _tmp['rent_price'] = helpers.scraper.digits_from_str(_tmp['rent_price']) if _tmp['rent_price'] is not None else None
        _tmp['gallery'] = json.loads(_tmp['gallery'])
        _tmp['tracking_id'] = int(helpers.scraper.digits_from_str(_tmp['tracking_id'])) if _tmp['tracking_id'] is not None else None
        _tmp = helpers.scraper.dict_except(_tmp,[],include_keys=columns)
        
        doc_list += [_tmp]

    gc.collect()
    return doc_list
    

In [14]:
ID_list = [i[:-5] for i in os.listdir(LOCAL_DATA_PATH) if i.endswith(".bson")]

In [28]:
%%time
raw_data = load_raw_data(ID_list, LOCAL_DATA_PATH)

CPU times: user 24.6 s, sys: 6.36 s, total: 30.9 s
Wall time: 41.1 s


In [29]:
%%time
db.insert_many(raw_data)

CPU times: user 4.36 s, sys: 3.58 s, total: 7.95 s
Wall time: 5min 23s


# Prepare data

In [ ]:
def list_mongo(db_):
    return [i['_id'] for i in db_.find({},{"_id":1})]

In [ ]:
def write_mongo(object_file, db_):
    try:
        return db_.update_one({'_id': object_file["_id"]}, {"$set": object_file}, upsert=True)
    except (ConnectionFailure, ServerSelectionTimeoutError) as e:
        logger.error("pymongo.errors, Could not connect to server: %s" % e)
    except (WriteError, WriteConcernError, WTimeoutError) as e:
        logger.error("pymongo.errors, Write error: %s" % e)
    except BaseException as e:
        logger.error("BaseException, something went wrong: %s" % e)
        
    

In [ ]:
ID = list_mongo(db)

In [ ]:
write_mongo(raw_data[0],db)

In [ ]:
import sys
sys.path.append("/Users/xszpo/Google Drive/DataScience/Projects/201907_xFlat_AWS_Scrapy/app_webscr_otodom")
import helpers as helpers
import gc
import numpy as np
import json

def mongo_otodom_transf_json(json_data):
    
    columns = ['_id','name','location','flat_size', 'rooms','floor','price',
               'price_m2', 'market', 'number_of_floors', 'floor_attic','floor_basement','building_type', 
               'building_material', 'widows_type', 'heating_type', 'year_of_building','finishing_stage', 
               'rent_price', 'property_form', 'available_from','description','additional_info',
               'GC_latitude','GC_longitude', 'GC_boundingbox', 'GC_addr_house_number','GC_addr_road', 
               'GC_addr_neighbourhood', 'GC_addr_suburb','GC_addr_city', 'GC_addr_county', 'GC_addr_state', 
               'GC_addr_postcode','GC_addr_country', 'GC_addr_country_code','url','main_url','gallery',
               'img_gallery_strimg']
    
    _tmp = json_data.copy()

    # PUT DATA INTO COLUMNS
    _tmp['_id'] = name        
    _tmp['GC_latitude'] = float(_tmp['geo_coordinates']['latitude'])
    _tmp['GC_longitude'] = float(_tmp['geo_coordinates']['longitude'])
    _tmp['GC_boundingbox'] = _tmp['geo_address_coordin']['@boundingbox']
    _tmp['GC_addr_house_number'] = _tmp['geo_address_text']['house_number'] if 'house_number' in _tmp['geo_address_text'] else None
    _tmp['GC_addr_road'] = _tmp['geo_address_text']['road'] if 'road' in _tmp['geo_address_text'] else None
    _tmp['GC_addr_neighbourhood'] = _tmp['geo_address_text']['neighbourhood'] if 'neighbourhood' in _tmp['geo_address_text'] else None
    _tmp['GC_addr_suburb'] = _tmp['geo_address_text']['suburb'] if 'suburb' in _tmp['geo_address_text'] else None
    _tmp['GC_addr_city'] = _tmp['geo_address_text']['city'] if 'city' in _tmp['geo_address_text'] else None
    _tmp['GC_addr_county'] = _tmp['geo_address_text']['county'] if 'country' in _tmp['geo_address_text'] else None
    _tmp['GC_addr_state'] = _tmp['geo_address_text']['state'] if 'state' in _tmp['geo_address_text'] else None
    _tmp['GC_addr_postcode'] = _tmp['geo_address_text']['postcode'] if 'postcode' in _tmp['geo_address_text'] else None
    _tmp['GC_addr_country'] = _tmp['geo_address_text']['country'] if 'country' in _tmp['geo_address_text'] else None
    _tmp['GC_addr_country_code'] = _tmp['geo_address_text']['country_code'] if 'country_code' in _tmp['geo_address_text'] else None
    _tmp['additional_info'] =_tmp['additional_info'].split("|")
    _ = _tmp.pop('geo_coordinates')
    _ = _tmp.pop('geo_address_coordin') 
    _ = _tmp.pop('price_per_square')

        # MODIFY DATA
    _tmp['flat_size'] = float(np.float32(helpers.scraper.digits_from_str(_tmp['flat_size']))) if _tmp['flat_size'] is not None else None
    _tmp['price'] = float(np.float32(helpers.scraper.digits_from_str(_tmp['price']))) if _tmp['price'] is not None else None
    _tmp['price_m2'] = helpers.scraper.digits_from_str(_tmp['price_m2']) if _tmp['price_m2'] is not None else None
    _tmp['rooms'] = int(helpers.scraper.digits_from_str(_tmp['rooms'])) if _tmp['rooms'] is not None else None
    _tmp['floor_attic'] = 1 if _tmp['floor']=='poddasze' else 0
    _tmp['floor_basement'] = 1 if _tmp['floor']=='suterena' else 0
    _tmp['floor'] = float(np.float32(helpers.scraper.convert_floor(_tmp['floor']))) if isinstance(_tmp['floor'], (str)) else None
    _tmp['number_of_floors'] = float(np.float32(_tmp['number_of_floors'])) if isinstance(_tmp['number_of_floors'], (str)) else None
    _tmp['year_of_building'] = float(np.float32(_tmp['year_of_building'])) if isinstance(_tmp['year_of_building'], (str)) else None
    _tmp['rent_price'] = helpers.scraper.digits_from_str(_tmp['rent_price']) if _tmp['rent_price'] is not None else None
    _tmp['gallery'] = json.loads(_tmp['gallery'])
    _tmp = helpers.scraper.dict_except(_tmp,[],include_keys=columns)
        
    gc.collect()
    return _tmp
    